In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer#MinMaxScaler, QuantileTransformer

In [2]:
game = pd.read_csv('games.csv')
#prefer price in USD, add describer
game = game.rename(columns = {'Price' : 'Price in USD'})
#drop some NA values + Playtests
#dropna isn't working for some reason, so I have to do drops like this
nannameindex = game.loc[game.Name.isna()].index
game.drop(nannameindex, inplace = True)
nangenindex = game.loc[game.Genres.isna()].index
game.drop(nangenindex, inplace = True)
game.drop(game.loc[game.Name.str.contains('Playtest')].index, inplace = True)
#drop unneeded tags
game.drop(columns = ['DLC count', 'About the game','Supported languages', 'Full audio languages','Score rank','Reviews', 'Header image', 'Website', 'Support url', 'Support email', 'Metacritic url', 'Achievements', 'Recommendations','Notes','Screenshots', 'Movies', 'Developers', 'Publishers', 'Categories', 'Metacritic score', 'Peak CCU'], inplace = True)
#drop software with the Utilities tag
game.drop(game.loc[game['Genres'].str.contains('Utilities', na = False)].index, inplace = True)

In [3]:
#Make large numbers readable
replace_values = {'0 - 0' : '0','0 - 20000' : '20,000 and less', '20000 - 50000' : '20,000 to 50,000', '50000 - 100000' : '50,000 to 100,000','100000 - 200000' : '100,000 to 200,000', '200000 - 500000' : '200,000 to 500,000', '500000 - 1000000' : '500,000 to 1 million','1000000 - 2000000' : '1 million to 2 million','2000000 - 5000000' : '2 million to 5 million', '5000000 - 10000000' : '5 million to 10 million', '10000000 - 20000000' : '10 million to 20 million', '20000000 - 50000000': '20 million to 50 million', '50000000 - 100000000' : '50 million to 100 million', '100000000 - 200000000' : '100 million to 200 million' }
game.replace({'Estimated owners' : replace_values}, inplace = True)
#Change Release date to datetime format
game['Release date'].replace(',', '', regex = True, inplace = True)
game['Release date'] = pd.to_datetime(game['Release date'])

In [4]:
#identify how many games qualify for each genre
genre_list = game.Genres
genre_list = genre_list.str.split(',', expand = True)
glist = []
for cols in genre_list.columns:
    tlist = genre_list[cols].value_counts().to_frame(cols).sort_index().reset_index()
    if cols == 0:
        glist = tlist
    else:
        glist = glist.merge(tlist, on = 'index', how = 'left')
glist['Sum'] = glist.sum(axis = 1, numeric_only = True)
glist.sort_values(by = 'Sum', ascending = False)
glist.rename(columns = {'index' : 'genres'}, inplace = True)

In [5]:
#drop non-game software
gcols = glist[glist['Sum']< 1000]
drop_check = game[['AppID', 'Genres']]

In [6]:
#save game releated genre tags
savelist = ("Nudity", "Sexual Content", "Violent", "Education", "Gore")
for item in savelist:
    temp = gcols.loc[gcols['genres'].str.contains(str(item))].index
    gcols.drop(temp, inplace = True)

C:\Users\John\AppData\Local\Temp\ipykernel_27452\2460550307.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gcols.drop(temp, inplace = True)


In [7]:
drop_list = pd.DataFrame()
for genre in gcols.genres:
    temp = drop_check.loc[drop_check.Genres.str.strip().str.contains(genre, case = False)]
    drop_list = pd.concat([drop_list, temp], ignore_index = True)

In [8]:
#use boolean indexing and invert to remove undesired genres
game = game[~game.AppID.isin(drop_list.AppID)]

In [9]:
#final null checking
for col in game.columns:
    if game[col].isnull().any() == True:
        print(col)

Tags


In [10]:
game

,AppID,Name,Release date,Estimated owners,Required age,Price in USD,Windows,Mac,Linux,User score,Positive,Negative,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Genres,Tags
0,20200,Galactic Bowling,2008-10-21,"20,000 and less",0,19.99,True,False,False,0,6,11,0,0,0,0,"Casual,Indie,Sports","Indie,Casual,Sports,Bowling"
1,655370,Train Bandit,2017-10-12,"20,000 and less",0,0.99,True,True,False,0,53,5,0,0,0,0,"Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc..."
2,1732930,Jolt Project,2021-11-17,"20,000 and less",0,4.99,True,False,False,0,0,0,0,0,0,0,"Action,Adventure,Indie,Strategy",NaN
3,1355720,Henosis™,2020-07-23,"20,000 and less",0,5.99,True,True,True,0,3,0,0,0,0,0,"Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz..."
4,1139950,Two Weeks in Painland,2020-02-03,"20,000 and less",0,0.00,True,True,False,0,50,8,0,0,0,0,"Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85098,2669080,Mannerheim's Saloon Car,2024-01-02,0,0,0.00,True,False,False,0,0,0,0,0,0,0,"Adventure,Simulation",NaN
85099,2736910,Beer Run,2024-01-03,0,0,0.00,True,False,False,0,0,0,0,0,0,0,"Casual,Indie",NaN
85100,2743220,My Friend The Spider,2024-01-04,0,0,0.00,True,False,False,0,0,0,0,0,0,0,"Adventure,Simulation",NaN
85101,2293130,Path of Survivors,2024-01-08,0,0,3.99,True,False,False,0,0,0,0,0,0,0,"Action,Casual,Indie,RPG,Simulation",NaN


In [11]:
review_score = pd.DataFrame(game[["AppID","Positive", "Negative" ]])

In [12]:
review_score['Total Reviews'] = review_score['Positive'] + review_score['Negative']

In [13]:
review_score['Pos_Percent'] = review_score['Positive'] / (review_score['Total Reviews'])

In [14]:
review_score['Pos_Percent'].describe()

count    62729.000000
mean         0.750740
std          0.247202
min          0.000000
25%          0.626374
50%          0.813559
75%          0.949367
max          1.000000
Name: Pos_Percent, dtype: float64

In [15]:
review_score['Pos_Percent'] = review_score['Pos_Percent'].fillna(0)

In [16]:
review_score

,AppID,Positive,Negative,Total Reviews,Pos_Percent
0,20200,6,11,17,0.352941
1,655370,53,5,58,0.913793
2,1732930,0,0,0,0.000000
3,1355720,3,0,3,1.000000
4,1139950,50,8,58,0.862069
...,...,...,...,...,...
85098,2669080,0,0,0,0.000000
85099,2736910,0,0,0,0.000000
85100,2743220,0,0,0,0.000000
85101,2293130,0,0,0,0.000000


In [17]:
review_score.sort_values(by = "Total Reviews", ascending = True)

,AppID,Positive,Negative,Total Reviews,Pos_Percent
85102,2738840,0,0,0,0.000000
69252,2247750,0,0,0,0.000000
69254,2291220,0,0,0,0.000000
69256,2265650,0,0,0,0.000000
69257,2240600,0,0,0,0.000000
...,...,...,...,...,...
8256,359550,929372,138530,1067902,0.870278
57205,271590,1171197,210154,1381351,0.847863
17585,570,1477153,300437,1777590,0.830986
8885,578080,1154655,895978,2050633,0.563072


In [18]:
review_score.loc[review_score['Total Reviews'] == 0]

,AppID,Positive,Negative,Total Reviews,Pos_Percent
2,1732930,0,0,0,0.0
7,1968760,0,0,0,0.0
18,1097880,0,0,0,0.0
29,1177420,0,0,0,0.0
43,1056520,0,0,0,0.0
...,...,...,...,...,...
85098,2669080,0,0,0,0.0
85099,2736910,0,0,0,0.0
85100,2743220,0,0,0,0.0
85101,2293130,0,0,0,0.0


In [19]:
game.AppID.isin(review_score.loc[review_score['Total Reviews'] == 0].AppID)

0        False
1        False
2         True
3        False
4        False
         ...  
85098     True
85099     True
85100     True
85101     True
85102     True
Name: AppID, Length: 80133, dtype: bool

In [20]:
#remove games with no reviews
game = game[~game.AppID.isin(review_score.loc[review_score['Total Reviews'] == 0].AppID)]

In [21]:
game.reset_index(drop=True, inplace = True)

In [22]:
review_score.Pos_Percent.describe(percentiles = [.2,.4,.6,.8])

count    80133.000000
mean         0.587688
std          0.379027
min          0.000000
20%          0.000000
40%          0.606643
50%          0.724005
60%          0.806452
80%          0.946423
max          1.000000
Name: Pos_Percent, dtype: float64

todo: use pos_percent to separate games into 5 groups, 1-5, where 5 = 1 and 1 = 0
score out of 10 for final?
fix multiilabel binarizer so each genre is its own column

In [23]:
genre_sep = pd.DataFrame(game[['AppID', 'Genres']])

In [24]:
cats = pd.DataFrame(genre_sep.Genres.apply(lambda x: pd.value_counts(x.split(","))).sum(axis = 0), columns = ['Count'])

In [25]:
#basic one-hot encoding for genre matching
#tofix - get 13 columns for 13 genres
cat_count = pd.get_dummies(genre_sep.Genres, prefix_sep = ',')

In [26]:
genre_sep.Genres = genre_sep.Genres.str.split(",")

In [27]:
genre_sep.Genres

0                                  [Casual, Indie, Sports]
1                                          [Action, Indie]
2                               [Adventure, Casual, Indie]
3                                       [Adventure, Indie]
4        [Adventure, Casual, Free to Play, Massively Mu...
                               ...                        
62724            [Casual, Indie, Simulation, Early Access]
62725                        [Casual, Indie, Early Access]
62726                           [Adventure, Casual, Indie]
62727                                   [Adventure, Indie]
62728                                      [Casual, Indie]
Name: Genres, Length: 62729, dtype: object

In [28]:
MLB = MultiLabelBinarizer(sparse_output = True)


In [29]:
f1 = genre_sep.Genres

In [30]:
MLB.fit(f1)
MLB.transform(f1)
cats_3 = pd.DataFrame.sparse.from_spmatrix(MLB.transform(f1), index = genre_sep.index, columns = MLB.classes_)

In [35]:
cats_3

,Action,Adventure,Casual,Early Access,Education,Free to Play,Gore,Indie,Massively Multiplayer,Movie,Nudity,RPG,Racing,Sexual Content,Simulation,Sports,Strategy,Violent
0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62724,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0
62725,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
62726,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
62727,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [32]:
genre_sep.drop("Genres", axis = 1, inplace = True)

In [36]:
genre_sep

,AppID
0,20200
1,655370
2,1355720
3,1139950
4,1469160
...,...
62724,2704060
62725,2645600
62726,2464700
62727,2602790


In [38]:
genre_sep = pd.concat([genre_sep, cats_3], axis = 1)

In [39]:
genre_sep

,AppID,Action,Adventure,Casual,Early Access,Education,Free to Play,Gore,Indie,Massively Multiplayer,Movie,Nudity,RPG,Racing,Sexual Content,Simulation,Sports,Strategy,Violent
0,20200,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
1,655370,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,1355720,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,1139950,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,1469160,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62724,2704060,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0
62725,2645600,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
62726,2464700,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
62727,2602790,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
